In [126]:
import re
import numpy as np
import pandas as pd

In [127]:
# %% load dataset
df = pd.read_csv('listings.csv.gz', compression='gzip')
print(df.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4601 entries, 0 to 4600
Data columns (total 106 columns):
id                                              int64
listing_url                                     object
scrape_id                                       int64
last_scraped                                    object
name                                            object
summary                                         object
space                                           object
description                                     object
experiences_offered                             object
neighborhood_overview                           object
notes                                           object
transit                                         object
access                                          object
interaction                                     object
house_rules                                     object
thumbnail_url                                   float64
medium_url   

In [128]:
df.head(5)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,360,https://www.airbnb.com/rooms/360,20190930152238,2019-09-30,LoHi Secret garden at the Chickadee Cottage,Come enjoy our oasis is the city and stay at o...,Chickadee Cottage is the largest of our guest ...,Come enjoy our oasis is the city and stay at o...,none,those who are interested in our local brews - ...,...,t,f,moderate,t,t,2,2,0,0,5.94
1,590,https://www.airbnb.com/rooms/590,20190930152238,2019-09-30,Comfortable - and a great value!,"Large guest room in my home, where I also live...",I have been enjoying welcoming many wonderful ...,"Large guest room in my home, where I also live...",none,I love the diversity of my neighborhood and it...,...,f,f,flexible,f,f,2,0,2,0,4.57
2,592,https://www.airbnb.com/rooms/592,20190930152238,2019-09-30,private,This room is in the basement. It does not hav...,This is a basement room. You can sometimes he...,This room is in the basement. It does not hav...,none,NaN,...,f,f,flexible,f,f,2,0,2,0,1.36
3,1940,https://www.airbnb.com/rooms/1940,20190930152238,2019-09-30,Baker Studio Close to EVERYTHING,Great place for a few nights or months! Signif...,"This newly built, highly functional studio is ...",Great place for a few nights or months! Signif...,none,Walking through the Baker historical neighborh...,...,f,f,strict_14_with_grace_period,f,f,1,1,0,0,1.26
4,2086,https://www.airbnb.com/rooms/2086,20190930152238,2019-09-30,Garden Level Condo,"A furnished, garden level, one bedroom/one bat...",Furnished one bedroom condo. Pets with additio...,"A furnished, garden level, one bedroom/one bat...",none,restaurants and park within walking distance. ...,...,f,f,strict_14_with_grace_period,f,f,1,1,0,0,0.63


In [129]:
df['host_since']

0       2008-07-08
1       2008-07-21
2       2008-07-21
3       2008-08-16
4       2008-08-19
5       2010-05-30
6       2008-07-08
7       2010-10-19
8       2008-08-07
9       2011-02-10
10      2010-03-25
11      2010-11-24
12      2010-11-24
13      2011-06-12
14      2011-04-21
15             NaN
16      2011-08-04
17      2009-03-19
18      2011-10-02
19      2011-12-05
20      2011-09-13
21      2011-08-06
22      2010-10-24
23      2009-03-21
24      2012-06-06
25      2011-10-20
26      2012-07-04
27      2012-08-01
28      2012-04-17
29      2012-08-22
           ...    
4571    2019-05-29
4572    2019-09-24
4573    2017-09-20
4574    2017-09-20
4575    2013-01-10
4576    2016-02-11
4577    2016-05-16
4578    2015-11-21
4579    2019-02-11
4580    2018-03-23
4581    2015-12-21
4582    2015-12-21
4583    2016-01-11
4584    2019-09-27
4585    2019-09-27
4586    2019-03-01
4587    2016-01-11
4588    2015-06-16
4589    2015-12-11
4590    2019-09-01
4591    2017-06-10
4592    2016

In [130]:
# %% preprocessing
# 1. time of host join Airbnb, processed to number days since first hosting in days
df_temp = (pd.to_datetime('2019-09-22') - pd.to_datetime(df['host_since'])).dt.days
df_select_X = df_temp.copy()

In [131]:
df_temp.head(5)

0    4093.0
1    4080.0
2    4080.0
3    4054.0
4    4051.0
Name: host_since, dtype: float64

In [132]:
df['host_response_time'].head(5)

0    within an hour
1    within an hour
2    within an hour
3    within an hour
4    within an hour
Name: host_response_time, dtype: object

In [133]:
# 2. host response time in categories, processed to 4 dummy variables
# null value exists, not included in any category
df_temp = pd.get_dummies(df[['host_response_time']], prefix='hrt')
df_select_X = pd.concat([df_select_X, df_temp], axis=1)
print(type(df_select_X))

<class 'pandas.core.frame.DataFrame'>


In [134]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour
0,4093.0,0,0,0,1
1,4080.0,0,0,0,1
2,4080.0,0,0,0,1
3,4054.0,0,0,0,1
4,4051.0,0,0,0,1


In [135]:
df['host_response_rate'].head(5)

0    100%
1    100%
2    100%
3    100%
4    100%
Name: host_response_rate, dtype: object

In [136]:
# 3. host response rate in percentage, processed to float [0, 1]
# null value exists, not included in any category
df_temp = df['host_response_rate'].astype('str').map(lambda x: float(x.strip('%')) / 100)
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [137]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate
0,4093.0,0,0,0,1,1.0
1,4080.0,0,0,0,1,1.0
2,4080.0,0,0,0,1,1.0
3,4054.0,0,0,0,1,1.0
4,4051.0,0,0,0,1,1.0


In [138]:
# 4. whether host is superhost, processed to dummy variable 1-True 0-False
df_temp = (df['host_is_superhost'] == 't').astype('int')
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [139]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost
0,4093.0,0,0,0,1,1.0,1
1,4080.0,0,0,0,1,1.0,1
2,4080.0,0,0,0,1,1.0,1
3,4054.0,0,0,0,1,1.0,1
4,4051.0,0,0,0,1,1.0,0


In [140]:
# 5. host listing count
df_select_X = pd.concat([df_select_X, df['host_listings_count']], axis=1)

In [141]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count
0,4093.0,0,0,0,1,1.0,1,3.0
1,4080.0,0,0,0,1,1.0,1,2.0
2,4080.0,0,0,0,1,1.0,1,2.0
3,4054.0,0,0,0,1,1.0,1,3.0
4,4051.0,0,0,0,1,1.0,0,1.0


In [142]:
# 6. whether host has profile picture, processed to dummy variable 1-True 0-False
df_temp = (df['host_has_profile_pic'] == 't').astype('int')
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [143]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic
0,4093.0,0,0,0,1,1.0,1,3.0,1
1,4080.0,0,0,0,1,1.0,1,2.0,1
2,4080.0,0,0,0,1,1.0,1,2.0,1
3,4054.0,0,0,0,1,1.0,1,3.0,1
4,4051.0,0,0,0,1,1.0,0,1.0,1


In [144]:
# 7. whether host identity is verified, processed to dummy variable 1-True 0-False
df_temp = (df['host_identity_verified'] == 't').astype('int')
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [145]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified
0,4093.0,0,0,0,1,1.0,1,3.0,1,1
1,4080.0,0,0,0,1,1.0,1,2.0,1,1
2,4080.0,0,0,0,1,1.0,1,2.0,1,1
3,4054.0,0,0,0,1,1.0,1,3.0,1,1
4,4051.0,0,0,0,1,1.0,0,1.0,1,1


In [146]:
# 8. host neighbourhood, processed to 26 dummy variables
df_temp = pd.get_dummies(df[['neighbourhood_cleansed']], prefix='nc')
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [147]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,nc_Virginia Village,nc_Washington Park,nc_Washington Park West,nc_Washington Virginia Vale,nc_Wellshire,nc_West Colfax,nc_West Highland,nc_Westwood,nc_Whittier,nc_Windsor
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [148]:
# 9. whether host location is exact, , processed to dummy variable 1-True 0-False
df_temp = (df['is_location_exact'] == 't').astype('int')
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [149]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,nc_Washington Park,nc_Washington Park West,nc_Washington Virginia Vale,nc_Wellshire,nc_West Colfax,nc_West Highland,nc_Westwood,nc_Whittier,nc_Windsor,is_location_exact
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,0,0,0,0,0,0,0,1
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,0,0,0,0,0,0,0,1
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,0,0,0,0,0,0,0,1
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,0,0,0,0,0,0,0,1
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,0,0,0,0,0,0,0,0,0,1


In [150]:
# 10. property_type, processed to 22 dummy variables
df_temp = pd.get_dummies(df[['property_type']], prefix='pt')
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [151]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,pt_Guesthouse,pt_Hostel,pt_House,pt_Loft,pt_Other,pt_Serviced apartment,pt_Tent,pt_Tiny house,pt_Townhouse,pt_Villa
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,1,0,0,0,0,0,0,0,0,0
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,1,0,0,0,0,0,0,0
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,1,0,0,0,0,0,0,0
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,1,0,0,0,0,0,0,0,0,0
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [152]:
# 11. room type, , processed to 4 dummy variables
df_temp = pd.get_dummies(df[['room_type']], prefix='rt')
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [153]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,pt_Other,pt_Serviced apartment,pt_Tent,pt_Tiny house,pt_Townhouse,pt_Villa,rt_Entire home/apt,rt_Hotel room,rt_Private room,rt_Shared room
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,0,0,0,0,1,0,0,0
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,0,0,0,0,0,0,1,0
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,0,0,0,0,0,0,1,0
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,0,0,0,0,1,0,0,0
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,0,0,0,0,0,0,1,0,0,0


In [154]:
# 12. maximum number of people the property accommodates
df_select_X = pd.concat([df_select_X, df['accommodates']], axis=1)

In [155]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,pt_Serviced apartment,pt_Tent,pt_Tiny house,pt_Townhouse,pt_Villa,rt_Entire home/apt,rt_Hotel room,rt_Private room,rt_Shared room,accommodates
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,0,0,0,1,0,0,0,4
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,0,0,0,0,0,1,0,3
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,0,0,0,0,0,1,0,2
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,0,0,0,1,0,0,0,2
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,0,0,0,0,0,1,0,0,0,2


In [156]:
# 13. number of bathrooms available
df_select_X = pd.concat([df_select_X, df['bathrooms']], axis=1)

# 14. number of bedrooms available
df_select_X = pd.concat([df_select_X, df['bedrooms']], axis=1)

# 15. number of beds available
df_select_X = pd.concat([df_select_X, df['beds']], axis=1)

In [157]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,pt_Townhouse,pt_Villa,rt_Entire home/apt,rt_Hotel room,rt_Private room,rt_Shared room,accommodates,bathrooms,bedrooms,beds
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,1,0,0,0,4,1.0,2,3
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,0,0,1,0,3,1.0,1,1
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,0,0,1,0,2,1.0,1,1
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,1,0,0,0,2,1.0,0,1
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,0,0,1,0,0,0,2,1.0,1,1


In [158]:
# 16. bed type, processed to 5 dummy variables
df_temp = pd.get_dummies(df[['bed_type']], prefix='bt')
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [159]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,rt_Shared room,accommodates,bathrooms,bedrooms,beds,bt_Airbed,bt_Couch,bt_Futon,bt_Pull-out Sofa,bt_Real Bed
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,0,4,1.0,2,3,0,0,1,0,0
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,3,1.0,1,1,0,0,0,0,1
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,2,1.0,1,1,0,0,0,0,1
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,0,2,1.0,0,1,0,0,0,0,1
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,0,2,1.0,1,1,0,0,0,0,1


In [160]:
# 17. price per night in dollars
df_temp = df['price'].map(lambda x: re.compile(r'[^\d.]+').sub('', str(x))).astype('float')
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [161]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,accommodates,bathrooms,bedrooms,beds,bt_Airbed,bt_Couch,bt_Futon,bt_Pull-out Sofa,bt_Real Bed,price
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,4,1.0,2,3,0,0,1,0,0,140.0
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,3,1.0,1,1,0,0,0,0,1,61.0
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,2,1.0,1,1,0,0,0,0,1,42.0
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,2,1.0,0,1,0,0,0,0,1,95.0
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,2,1.0,1,1,0,0,0,0,1,76.0


In [162]:
# 18. price per week in dollars
df_temp = df['weekly_price'].copy()
df_temp.loc[df['weekly_price'].isnull()] = 99999
df_temp = df_temp.map(lambda x: re.compile(r'[^\d.]+').sub('', str(x))).astype('float')
df_temp.loc[df_temp == 99999] = np.nan
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [163]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,bathrooms,bedrooms,beds,bt_Airbed,bt_Couch,bt_Futon,bt_Pull-out Sofa,bt_Real Bed,price,weekly_price
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,1.0,2,3,0,0,1,0,0,140.0,NaN
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,1.0,1,1,0,0,0,0,1,61.0,375.0
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,1.0,1,1,0,0,0,0,1,42.0,295.0
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,1.0,0,1,0,0,0,0,1,95.0,NaN
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,1.0,1,1,0,0,0,0,1,76.0,NaN


In [164]:
#handle the roles whose weekly_price is NaN, fill in them by the mean value

In [165]:
# 19. price per month in dollars
df_temp = df['monthly_price'].copy()
df_temp.loc[df['monthly_price'].isnull()] = 99999
df_temp = df_temp.map(lambda x: re.compile(r'[^\d.]+').sub('', str(x))).astype('float')
df_temp.loc[df_temp == 99999] = np.nan
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [166]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,bedrooms,beds,bt_Airbed,bt_Couch,bt_Futon,bt_Pull-out Sofa,bt_Real Bed,price,weekly_price,monthly_price
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,2,3,0,0,1,0,0,140.0,NaN,NaN
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,1,1,0,0,0,0,1,61.0,375.0,1275.0
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,1,1,0,0,0,0,1,42.0,295.0,975.0
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,0,1,0,0,0,0,1,95.0,NaN,NaN
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,1,1,0,0,0,0,1,76.0,NaN,NaN


In [167]:
# 20. security deposit in dollars
df_temp = df['security_deposit'].copy()
df_temp.loc[df['security_deposit'].isnull()] = 99999
df_temp = df_temp.map(lambda x: re.compile(r'[^\d.]+').sub('', str(x))).astype('float')
df_temp.loc[df_temp == 99999] = np.nan
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [168]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,beds,bt_Airbed,bt_Couch,bt_Futon,bt_Pull-out Sofa,bt_Real Bed,price,weekly_price,monthly_price,security_deposit
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,3,0,0,1,0,0,140.0,NaN,NaN,150.0
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,1,0,0,0,0,1,61.0,375.0,1275.0,NaN
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,1,0,0,0,0,1,42.0,295.0,975.0,NaN
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,1,0,0,0,0,1,95.0,NaN,NaN,150.0
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,1,0,0,0,0,1,76.0,NaN,NaN,100.0


In [169]:
# 21. cleaning fee in dollars
df_temp = df['cleaning_fee'].copy()
df_temp.loc[df['cleaning_fee'].isnull()] = 99999
df_temp = df_temp.map(lambda x: re.compile(r'[^\d.]+').sub('', str(x))).astype('float')
df_temp.loc[df_temp == 99999] = np.nan
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [170]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,bt_Airbed,bt_Couch,bt_Futon,bt_Pull-out Sofa,bt_Real Bed,price,weekly_price,monthly_price,security_deposit,cleaning_fee
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,1,0,0,140.0,NaN,NaN,150.0,40.0
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,0,0,1,61.0,375.0,1275.0,NaN,10.0
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,0,0,1,42.0,295.0,975.0,NaN,10.0
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,0,0,1,95.0,NaN,NaN,150.0,75.0
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,0,0,0,0,1,76.0,NaN,NaN,100.0,30.0


In [171]:
# 22. number of guests allowed in initial listing
df_select_X = pd.concat([df_select_X, df['guests_included']], axis=1)

In [172]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,bt_Couch,bt_Futon,bt_Pull-out Sofa,bt_Real Bed,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,0,1,0,0,140.0,NaN,NaN,150.0,40.0,2
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,0,1,61.0,375.0,1275.0,NaN,10.0,1
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,0,1,42.0,295.0,975.0,NaN,10.0,1
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,0,1,95.0,NaN,NaN,150.0,75.0,2
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,0,0,0,1,76.0,NaN,NaN,100.0,30.0,1


In [173]:
# 23. additional fee for each additional guest in dollars
df_temp = df['extra_people'].map(lambda x: re.compile(r'[^\d.]+').sub('', str(x))).astype('float')
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [174]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,bt_Futon,bt_Pull-out Sofa,bt_Real Bed,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,1,0,0,140.0,NaN,NaN,150.0,40.0,2,15.0
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,1,61.0,375.0,1275.0,NaN,10.0,1,5.0
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,0,1,42.0,295.0,975.0,NaN,10.0,1,5.0
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,1,95.0,NaN,NaN,150.0,75.0,2,100.0
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,0,0,1,76.0,NaN,NaN,100.0,30.0,1,25.0


In [175]:
# 24. minimum number of nights needed for booking
df_select_X = pd.concat([df_select_X, df['minimum_nights']], axis=1)

In [176]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,bt_Pull-out Sofa,bt_Real Bed,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,0,0,140.0,NaN,NaN,150.0,40.0,2,15.0,1
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,1,61.0,375.0,1275.0,NaN,10.0,1,5.0,1
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,0,1,42.0,295.0,975.0,NaN,10.0,1,5.0,30
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,0,1,95.0,NaN,NaN,150.0,75.0,2,100.0,2
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,0,1,76.0,NaN,NaN,100.0,30.0,1,25.0,180


In [177]:
# 25. maximum number of nights needed for booking
df_select_X = pd.concat([df_select_X, df['maximum_nights']], axis=1)

In [178]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,bt_Real Bed,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,0,140.0,NaN,NaN,150.0,40.0,2,15.0,1,29
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,1,61.0,375.0,1275.0,NaN,10.0,1,5.0,1,300
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,1,42.0,295.0,975.0,NaN,10.0,1,5.0,30,365
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,1,95.0,NaN,NaN,150.0,75.0,2,100.0,2,120
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,1,76.0,NaN,NaN,100.0,30.0,1,25.0,180,1125


In [179]:
# 26. whether the host shows the availability of the listing, processed to dummy variable 1-True 0-False
df_temp = (df['has_availability'] == 't').astype('int')
df_select_X = pd.concat([df_select_X, df_temp], axis=1)

In [180]:
df_select_X.head(5)

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,...,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,has_availability
0,4093.0,0,0,0,1,1.0,1,3.0,1,1,...,140.0,NaN,NaN,150.0,40.0,2,15.0,1,29,1
1,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,61.0,375.0,1275.0,NaN,10.0,1,5.0,1,300,1
2,4080.0,0,0,0,1,1.0,1,2.0,1,1,...,42.0,295.0,975.0,NaN,10.0,1,5.0,30,365,1
3,4054.0,0,0,0,1,1.0,1,3.0,1,1,...,95.0,NaN,NaN,150.0,75.0,2,100.0,2,120,1
4,4051.0,0,0,0,1,1.0,0,1.0,1,1,...,76.0,NaN,NaN,100.0,30.0,1,25.0,180,1125,1


In [181]:
# %% target variables
# 1. number of days available for booking in the next 30, 60, 90, 365 days
df_select_Y = df[['availability_30', 'availability_60', 'availability_90', 'availability_365']].copy()

In [182]:
df_select_Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4601 entries, 0 to 4600
Data columns (total 4 columns):
availability_30     4601 non-null int64
availability_60     4601 non-null int64
availability_90     4601 non-null int64
availability_365    4601 non-null int64
dtypes: int64(4)
memory usage: 143.9 KB


In [183]:
df_select_X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4601 entries, 0 to 4600
Data columns (total 131 columns):
host_since                         float64
hrt_a few days or more             uint8
hrt_within a day                   uint8
hrt_within a few hours             uint8
hrt_within an hour                 uint8
host_response_rate                 float64
host_is_superhost                  int64
host_listings_count                float64
host_has_profile_pic               int64
host_identity_verified             int64
nc_Athmar Park                     uint8
nc_Auraria                         uint8
nc_Baker                           uint8
nc_Barnum                          uint8
nc_Barnum West                     uint8
nc_Bear Valley                     uint8
nc_Belcaro                         uint8
nc_Berkeley                        uint8
nc_CBD                             uint8
nc_Capitol Hill                    uint8
nc_Chaffee Park                    uint8
nc_Cheesman Park         

In [184]:
print(df_select_X.isnull().sum())

host_since                           4
hrt_a few days or more               0
hrt_within a day                     0
hrt_within a few hours               0
hrt_within an hour                   0
host_response_rate                 591
host_is_superhost                    0
host_listings_count                  4
host_has_profile_pic                 0
host_identity_verified               0
nc_Athmar Park                       0
nc_Auraria                           0
nc_Baker                             0
nc_Barnum                            0
nc_Barnum West                       0
nc_Bear Valley                       0
nc_Belcaro                           0
nc_Berkeley                          0
nc_CBD                               0
nc_Capitol Hill                      0
nc_Chaffee Park                      0
nc_Cheesman Park                     0
nc_Cherry Creek                      0
nc_City Park                         0
nc_City Park West                    0
nc_Civic Center          

In [185]:
# analyze the condition of NaN value in each columns, and decides which columns need to be removed,
# in this eample we remove host_response_rate, weekly_price, monthly_price, security_deposit, cleaning_fee
df_select_X.drop(['host_response_rate'],axis=1,inplace=True)
df_select_X.drop(['weekly_price'],axis=1,inplace=True)
df_select_X.drop(['monthly_price'],axis=1,inplace=True)
df_select_X.drop(['security_deposit'],axis=1,inplace=True)
df_select_X.drop(['cleaning_fee'],axis=1,inplace=True)

In [192]:
df_select_X[df_select_X.isnull().values==True]#.index

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,nc_Athmar Park,...,bt_Couch,bt_Futon,bt_Pull-out Sofa,bt_Real Bed,price,guests_included,extra_people,minimum_nights,maximum_nights,has_availability
12,3224.0,0,1,0,0,1,3.0,1,1,0,...,0,0,0,1,60.0,1,0.0,2,60,1
15,NaN,0,0,0,0,0,NaN,0,0,0,...,0,0,0,1,35.0,1,0.0,30,360,1
15,NaN,0,0,0,0,0,NaN,0,0,0,...,0,0,0,1,35.0,1,0.0,30,360,1
19,2848.0,0,0,0,0,0,2.0,1,0,0,...,0,0,0,1,3800.0,4,25.0,30,365,1
186,1895.0,0,0,0,1,1,1.0,1,0,0,...,0,0,0,1,115.0,2,0.0,1,1125,1
521,NaN,0,0,0,0,0,NaN,0,0,0,...,0,0,0,1,40.0,1,0.0,30,365,1
521,NaN,0,0,0,0,0,NaN,0,0,0,...,0,0,0,1,40.0,1,0.0,30,365,1
2192,NaN,0,0,0,0,0,NaN,0,0,0,...,0,0,0,1,105.0,2,60.0,3,10,1
2192,NaN,0,0,0,0,0,NaN,0,0,0,...,0,0,0,1,105.0,2,60.0,3,10,1
3222,NaN,0,0,0,0,0,NaN,0,0,0,...,0,0,0,1,89.0,2,20.0,1,8,1


In [196]:
df_select_X.dropna(inplace=True)

In [197]:
#no rows contain NaN value any more
df_select_X[df_select_X.isnull().values==True]

,host_since,hrt_a few days or more,hrt_within a day,hrt_within a few hours,hrt_within an hour,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,nc_Athmar Park,...,bt_Couch,bt_Futon,bt_Pull-out Sofa,bt_Real Bed,price,guests_included,extra_people,minimum_nights,maximum_nights,has_availability


# Data Visualization